In [7]:
import serial 
import struct
import time

RETURN_TEMP   = 0x01
RETURN_SAMPLE = 0x02
RETURN_HELLO  = 0x03
SET_BIAS      = 0x04
RETURN_BATT   = 0x05
TOGGLE_RED    = 0x06

det_serialport = '/dev/ttyACM0' #if using the v0.1 board
xray_serialport = '/dev/ttyUSB0'

def getFloatValue(ser, cmd):
    assert(cmd == RETURN_TEMP or cmd == RETURN_SAMPLE or cmd == RETURN_BATT)
    ser.write(bytes([cmd, 0x00]))
    data = ser.read(4)
    return struct.unpack('>i', data)[0]/100

def testConnection(ser):
    ser.write(bytes([RETURN_HELLO, 0x00]))
    data = ser.read(8)
    return data.strip().decode('utf-8') 

def setBias(ser, bias):
    assert(bias >= 0 and bias <= 255)
    ser.write(bytes([SET_BIAS, bias]))
    data = ser.read(8)
    return data.strip().decode('utf-8') 

In [5]:
with serial.Serial(det_serialport, 115200, timeout=2) as ser:
  print(testConnection(ser))
  
  #loop to get batt voltage with 100ms delay
  for i in range(10):
    print(getFloatValue(ser, RETURN_BATT))
    time.sleep(0.1)
  

461s058:
19370069.62
19530665.99
5423984.96
19530663.41
19134572.04
9092104.16
9260386.14
11453238.34
5401613.32
2187725.34


In [5]:
with serial.Serial(det_serialport, 115200, timeout=2) as ser:
  #read the temperature in C, set the bias, and read the sipm, return bias to 0
  print(testConnection(ser))
  print(getFloatValue(ser, RETURN_TEMP))
  print(setBias(ser, 0x01))
  print(getFloatValue(ser, RETURN_SAMPLE))
  print(setBias(ser, 0))
  print(getFloatValue(ser, RETURN_BATT))

hi
27.62
OK
2961.93
OK
5.2


In [6]:
with serial.Serial(det_serialport, 115200, timeout=2) as ser:
  ser.write(bytes([TOGGLE_RED, 0x00]))

In [1]:
xray_serialport = '/dev/ttyUSB0'
from serial import Serial
import time

from XRS4_driver import XRS4
with Serial(xray_serialport, 57600, timeout=1) as xray_ser:
    xray = XRS4(xray_ser)
    xray.set_pulse_count(1)
    xray.fire_xray()
    xray.set_pulse_count(10)
    xray.fire_xray()


b'!ERR|0 - Invalid # command\n'
b'!VER|2.12\n'
b'!PLSCNT|1\n'
b'!MSG|Firing X-Ray\n'
b'!PULSE|1\n'
b'!PLSCNT|10\n'


In [5]:
import numpy as np
import serial
from datetime import datetime

from CZT import CZT

tempChamber = CZT()

timestamp = datetime.now()

Min_temp = -10
Max_temp = 60
Step_Temp = 10
Min_dose = 1
Max_dose = 25
Step_dose = 5
Min_bias = 5
Max_bias = 50
Step_bias = 5

Temp_Range = np.arange(Min_temp, Max_temp, Step_Temp)
Dose_Range = np.arange(Min_dose, Max_dose, Step_dose)
Bias_Range = np.arange(Min_bias, Max_bias, Step_bias)

data = np.zeros( (len(Temp_Range), len(Dose_Range), len(Bias_Range)) )

for i, temp in enumerate(Temp_Range):
    print(f'set temp {temp}')
    #setting the temperature
    tempChamber.setTemperatureSP(temp)
    #wait for temp at SiPM
    #with serial.Serial(serialport, 115200, timeout=2) as ser:
         #current_t = getFloatValue(ser, RETURN_TEMP)
    for j, dose in enumerate(Dose_Range):
        print(f' set dose {dose}')
        #dose
        for k, bias in enumerate(Bias_Range):
            #measure
            #with serial.Serial(serialport, 115200, timeout=2) as ser:
                #setBias(ser, bias)
                #figure out what bias actually means, what range is good
                #data[i,j,k] = getFloatValue(ser, RETURN_SAMPLE)
                print(f'  measure bias {bias}')
            

#with open(f'data_{timestamp}', data)

set temp -10
 set dose 1
  measure bias 5
  measure bias 10
  measure bias 15
  measure bias 20
  measure bias 25
  measure bias 30
  measure bias 35
  measure bias 40
  measure bias 45
 set dose 6
  measure bias 5
  measure bias 10
  measure bias 15
  measure bias 20
  measure bias 25
  measure bias 30
  measure bias 35
  measure bias 40
  measure bias 45
 set dose 11
  measure bias 5
  measure bias 10
  measure bias 15
  measure bias 20
  measure bias 25
  measure bias 30
  measure bias 35
  measure bias 40
  measure bias 45
 set dose 16
  measure bias 5
  measure bias 10
  measure bias 15
  measure bias 20
  measure bias 25
  measure bias 30
  measure bias 35
  measure bias 40
  measure bias 45
 set dose 21
  measure bias 5
  measure bias 10
  measure bias 15
  measure bias 20
  measure bias 25
  measure bias 30
  measure bias 35
  measure bias 40
  measure bias 45
set temp 0
 set dose 1
  measure bias 5
  measure bias 10
  measure bias 15
  measure bias 20
  measure bias 25
  measu